In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
!wget https://tvqa.cs.unc.edu/files/tvqa_plus_annotations.tar.gz
!tar -xvf tvqa_plus_annotations.tar.gz

--2022-04-03 05:01:19--  https://tvqa.cs.unc.edu/files/tvqa_plus_annotations.tar.gz
Resolving tvqa.cs.unc.edu (tvqa.cs.unc.edu)... 152.2.132.230
Connecting to tvqa.cs.unc.edu (tvqa.cs.unc.edu)|152.2.132.230|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6718915 (6.4M) [application/x-gzip]
Saving to: ‘tvqa_plus_annotations.tar.gz’

tvqa_plus_annotatio 100%[===================>]   6.41M  32.1MB/s    in 0.2s    

2022-04-03 05:01:22 (32.1 MB/s) - ‘tvqa_plus_annotations.tar.gz’ saved [6718915/6718915]

./._tvqa_plus_train.json
tvqa_plus_train.json
./._tvqa_plus_val.json
tvqa_plus_val.json


In [ ]:
!wget https://tvqa.cs.unc.edu/files/tvqa_plus_subtitles.tar.gz
!tar xvf tvqa_plus_subtitles.tar.gz

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!cp -r drive/MyDrive/Prediction_jsons/ .

In [19]:
#VIDEO UNI
uni_video = '/content/Prediction_jsons/VIDEO_UNI_PRED.json'
data = 'tvqa_plus_val.json'

#Subtitles UNI
uni_sub = '/content/Prediction_jsons/BERT_SUBTITLE_UNIMODAL/BERT_SUBTITLES_UNIMODAL_PRED.json'

#QA only
uni_qa = '/content/Prediction_jsons/QA_only_pred.json'

#Multimodal BERT RESNET LINEAR
mm_lin = '/content/Prediction_jsons/MM_BERT_RES_LIN_PRED.json'

#Multimodal GLOVE RESNET MULTIMODAL PRED
mm_glove = '/content/Prediction_jsons/glove_resnet_self_attention/GLOVE_RESNET_MULTIMODAL_PRED.JSON'

#Multimodal BERT RESNET 
mm_sa = '/content/Prediction_jsons/BERT_RESNET_SELF_ATT_MULTIMODAL/BERT_RESNET_MULTIMODAL_PRED.json'

In [21]:
def load_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f, encoding='utf-8')

In [22]:
models = [uni_video, uni_sub, uni_qa, mm_lin, mm_glove, mm_sa]
model_preds = []
for i in models:
  temp = load_json(i)
  predictions = {}
  for i in temp:
    predictions[i] = str(temp[i])
  model_preds.append(predictions)

In [27]:
val_data = load_json(data)

In [112]:
val_sub = load_json('/content/tvqa_plus_subtitles.json')

In [113]:
model_names = ['UNI_VIDEO', 'UNI_SUB', 'UNI_QA', 'MM_LIN', 'MM_GLOVE', 'MM_SA']
for i in [5, 978, 1589, 2546]:
  print("Question: ", val_data[i]['q'])
  print("Answers: ")
  print("a0: ", val_data[i]['a0'])
  print("a1: ", val_data[i]['a1'])
  print("a2: ", val_data[i]['a2'])
  print("a3: ", val_data[i]['a3'])
  print("a4: ", val_data[i]['a4'])
  print("Correct Answer: ", val_data[i]['a'+str(val_data[i]['answer_idx'])])
  print("Subtitle: ", val_sub[val_data[i]['vid_name']]['sub_text'])
  print("Predictions: ")
  for idx, model in enumerate(model_preds):
    pred_idx = model[str(val_data[i]['qid'])]
    print(model_names[idx], ": ", val_data[i]['a'+str(pred_idx)])
  print()

Question:  Where is Leonard sitting when Lesley talks about the kiss ?
Answers: 
a0:  Leonard is sitting on the sink .
a1:  Leonard is sitting on the barstool .
a2:  Leonard is sitting on the floor .
a3:  Leonard is sitting in Lesley 's lap .
a4:  Leonard is sitting behind the desk .
Correct Answer:  Leonard is sitting on the barstool .
Subtitle:  UNKNAME : What do you think ? You proposed the experiment . <eos> Leonard : I think you should present your findings first . <eos> Lesley : Fair enough . <eos> Lesley : On the plus side , it was a good kiss , reasonable technique . <eos> Lesley : . no extraneous spittle . <eos> Lesley : On the other hand , no arousal . <eos> UNKNAME : None ? None . <eos> Leonard : Ah . <eos> Leonard : Well , thank you for your time . <eos> Penny : Thank you . Hmm . <eos> Leonard : None at all ? <eos> Howard : Sheldon , if you were a robot and I knew and you did n't . <eos> Howard : . would you want me to tell you ? <eos> Sheldon : That depends . <eos> Sheldon

In [86]:
import os, sys
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request

#for bbt, the episodes don't have bbt in frames folder name
shows_prefix_link = {
    'bbt':'https://cardibmmml.s3.us-east-1.amazonaws.com/frames_hq/',
    'house':'https://tvqammml.s3.us-east-1.amazonaws.com/',
    'grey':'https://tvqammml.s3.us-east-1.amazonaws.com/',
    'castle':'https://cardibmmml.s3.us-east-1.amazonaws.com/frames_hq/',
    'met':'https://tvqammml.s3.us-east-1.amazonaws.com/',
    'friends':'https://cardibmmml.s3.us-east-1.amazonaws.com/frames_hq/',
}

def get_s3_name(link):
  start = link.index('://') + 3
  end = link.index('.s3')
  bucket_name = link[start:end]
  return bucket_name

def clear_frames_buffer():
  while (len(os.listdir('/content/frames_buffer')) != 0):
    os.system('rm -rf /content/frames_buffer/*')
  return

def get_frames(vid_name, skip=1):
  #skip=number of frames to be skipped during retrieval

  clear_frames_buffer()
  filename_dummy = '00000'
  _index = vid_name.index('_')
  show_name = vid_name[:_index]
  if (show_name not in shows_prefix_link):
    show_name = 'bbt'
  frames_folder = show_name + '_' + 'frames'
  print(show_name)
  prefix_link = ''
  if (get_s3_name(shows_prefix_link[show_name]) == 'tvqammml'):
    prefix_link = shows_prefix_link[show_name] 
  else:
    prefix_link = shows_prefix_link[show_name] + frames_folder + '/'
  for i in range(1,300,skip):
    filename_temp = (filename_dummy + str(i))[-5:] + '.jpg'
    frame_link = prefix_link + vid_name + '/' + filename_temp
    try:
      urllib.request.urlretrieve(frame_link, '/content/frames_buffer/'+filename_temp)
    except:
      print("Reached end.")
      if (len(os.listdir('/content/frames_buffer')) == 0):
        print("Frames not found.")
      break

[5, 978, 1589, 2546]

In [97]:
get_frames(val_data[2546]['vid_name'])

bbt
Reached end.
